In [1]:
import pandas as pd
# importem el service_account i bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

path_credentials = 'credentials.json' # Arxiu al gitingore per seguretat

# definim la variable amb les credencials
credentials = service_account.Credentials.from_service_account_file(
    path_credentials)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
df = pd.read_csv(r'archive\fda_approved_food_items_w_nutrient_info.csv', low_memory=False)
# Posem totes les columnes en minúscules, substituim les comes per - i els espais per _
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(',', '-')

C:\Users\MarcNosas\AppData\Local\Temp\ipykernel_24584\4020451839.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
C:\Users\MarcNosas\AppData\Local\Temp\ipykernel_24584\4020451839.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')


In [24]:
# Tots els camps del tipus string els posem en minúscules
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower()
    

In [29]:
# utilitzem regex per filtrar les dades agafaren les dades que al camp brand_owner tinguin target o wal?mart
df = df.dropna(subset=['brand_owner'])
df = df[df['brand_owner'].str.contains('(target|walmart|wal.mart)', regex=True)]


C:\Users\MarcNosas\AppData\Local\Temp\ipykernel_24584\3934648112.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['brand_owner'].str.contains('(target|walmart|wal.mart)', regex=True)]


In [3]:
# Agfem només els primers 1000 registres per a fer proves# passem el dataframe a un fitxer csv
df.to_csv('fda_approved_food_items_w_nutrient_info.csv', index=False)

In [31]:
df.serving_size_unit.unique()

array(['g', 'ml', nan, 'grm', 'iu', 'mlt', 'mg', 'gm', 'mc'], dtype=object)

In [36]:
# Farem una funció que rebrà els camps serving_size i serving_size_unit. 
# si tenim mg multiplicarem serving_size per 0.001, si tenim g no farem res

def serving_size(row):
    if row['serving_size_unit'] == 'mg':
        return row['serving_size'] * 0.001
    else:
        return row['serving_size']

df['serving_size'] = df.apply(serving_size, axis=1)

# Ara canviem les unitats de la columna serving_size_unit a g si és mg,g, grm o gm
df['serving_size_unit'] = df['serving_size_unit'].replace({'mg': 'g', 'g': 'g', 'grm': 'g', 'gm': 'g'})

# Ara tot allò que no sigui g ho filtrem
df = df[df['serving_size_unit'] == 'g']

In [37]:
df.shape

(76725, 45)

In [35]:
# mirem on serving_size_unit hi ha el valor gm
df[df['serving_size_unit'].isna()].head()


,fdc_id,brand_owner,description,ingredients,gtin_upc,serving_size,serving_size_unit,branded_food_category,modified_date,available_date,calcium_ca-mg,carbohydrate_by_difference-g,cholesterol-mg,copper_cu-mg,energy-kcal,fatty_acids_total_monounsaturated-g,fatty_acids_total_polyunsaturated-g,fatty_acids_total_saturated-g,fatty_acids_total_trans-g,fiber_soluble-g,fiber_total_dietary-g,folate_total-ug,folic_acid-ug,iron_fe-mg,magnesium_mg-mg,manganese_mn-mg,niacin-mg,pantothenic_acid-mg,phosphorus_p-mg,potassium_k-mg,protein-g,riboflavin-mg,sodium_na-mg,sugars_added-g,sugars_total_including_nlea-g,thiamin-mg,total_lipid_fat-g,total_sugar_alcohols-g,vitamin_a_iu-iu,vitamin_b-12-ug,vitamin_b-6-mg,vitamin_c_total_ascorbic_acid-mg,vitamin_d_d2_+_d3_international_units-iu,vitamin_d_d2_+_d3-ug,zinc_zn-mg
1458334,2198011,"wal-mart stores, inc.","deluxe american cheese slices, deluxe american","milk, cream, water, sodium citrate, salt, chee...",078742353272,19.0,NaN,cheese,2021-11-16,2021-12-30,316.0,5.26,105.0,NaN,368.0,NaN,NaN,18.42,0.0,NaN,0.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,15.79,NaN,1789.0,0.0,5.26,NaN,31.58,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1458338,2198015,"wal-mart stores, inc.","pear halves in heavy syrup, heavy syrup","pears, water, high fructose corn syrup, sugar,...",078742369655,128.0,NaN,canned fruit,2021-11-18,2021-12-30,4.0,19.53,0.0,NaN,78.0,NaN,NaN,0.00,0.0,NaN,1.6,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,42.0,0.00,NaN,4.0,10.9,16.41,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1458339,2198016,"wal-mart stores, inc.","turkey gravy mix, turkey","wheat flour, food starch-modified, corn syrup ...",078742371856,6.0,NaN,gravy mix,2021-11-22,2021-12-30,0.0,66.67,0.0,NaN,333.0,NaN,NaN,0.00,0.0,NaN,0.0,NaN,NaN,3.33,NaN,NaN,NaN,NaN,NaN,167.0,0.00,NaN,6000.0,0.0,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458484,2198162,"wal-mart stores, inc.","chicken gravy mix, chicken","wheat flour, corn starch, salt, dextrose, chic...",078742371849,6.0,NaN,gravy mix,2021-11-22,2021-12-30,167.0,50.00,0.0,NaN,333.0,NaN,NaN,0.00,0.0,NaN,0.0,NaN,NaN,1.67,NaN,NaN,NaN,NaN,NaN,333.0,16.67,NaN,5167.0,16.7,16.67,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458491,2198170,"wal-mart stores, inc.","pear halves in pear juice from concentrate, pe...","pears, pear juice (water, pear juice concentra...",078742370606,124.0,NaN,canned fruit,2021-11-16,2021-12-30,8.0,12.10,0.0,NaN,48.0,NaN,NaN,0.00,0.0,NaN,1.6,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,89.0,0.00,NaN,0.0,0.0,8.87,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [38]:
# Ara ja podem calcular el camp de densitat calòrica (KCal/g) i també calcularem un camp de si els ingredients porten sucre o no (booleà) i si consta en el camp "sugars\_added".

# Creem una funció que ens retorni el valor de la densitat calòrica
def densitat_calorica(row):
    return row['energy-kcal'] / row['serving_size']

df['cal_denisty'] = df.apply(densitat_calorica, axis=1)

# I ara mirarem si els ingredients porten sucre o no
# utilitzem regex per filtrar les dades agafaren les dades que al camp ingredients tinguin sugar o sweetener
df['contains_sugar'] = df['ingredients'].str.contains('(sugar|sweetener)', regex=True)

# Fem el mateix amb el camp sugars_added, si sugars_added-g és més gran que 0 posarem True, si no False
df['sugars_added'] = df['sugars_added-g'] > 0


C:\Users\MarcNosas\AppData\Local\Temp\ipykernel_24584\1579965343.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['contains_sugar'] = df['ingredients'].str.contains('(sugar|sweetener)', regex=True)


In [40]:
# El camp serving_size ha perdut el sentit
df = df.drop(columns=['serving_size_unit'])

# i canviem el nom de la columna serving_size per serving_size_g
df = df.rename(columns={'serving_size': 'serving_size-g'})

In [41]:
# Guardem el fitxer
df.to_csv('fda_approved_food_items_w_nutrient_info_clean.csv', index=False)